## 카카오맵에서 천안시 카페 크롤링하는 코드

In [1]:
import time
import json
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys

In [2]:
url = "https://map.kakao.com"

driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)
print(driver.current_url)

https://map.kakao.com/


In [3]:
search_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "search.keyword.query"))
    )
search_input.send_keys("천안 카페")
time.sleep(3)

dimmed_layer = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "dimmedLayer"))
    )
dimmed_layer.click()

search_button = driver.find_element(By.ID, "search.keyword.submit")
search_button.click()

time.sleep(3)

In [4]:
more_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "info.search.place.more"))
    )
more_button.click()
time.sleep(7)

In [5]:
import re
def collect_data_from_page(page_number):
    try:
        page_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, f"info.search.page.no{page_number}"))
        )
        page_link.click()
        
        time.sleep(5)

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        li_tags = soup.find_all('li', class_=re.compile('.*PlaceItem clickArea.*')) 
        # PlaceItem clickArea만 하면 하나씩 놓쳐서 re로 정규식 표현으로 그걸 포함하게 만들어주기
        
        print(len(li_tags)) # 디버깅으로 li 제대로 잘 잡히는지 확인
        
        for li in li_tags:
            title_tag = li.find('span', {'data-id': 'screenOutName'})
            address_tag = li.find('p', {'data-id': 'address'})
            if title_tag and address_tag:
                title = title_tag.get_text(strip=True)
                address = address_tag.get_text(strip=True)
                title_list.append(title)
                address_list.append(address)
    except Exception as e:
        print(e)

def click_next_button():
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "info.search.page.next"))
        )
        next_button.click()
        time.sleep(5) 
    except Exception as e:
        print(e)


In [6]:
title_list = []
address_list = []

In [7]:
for page_num in (range(1, 6)):  
    collect_data_from_page(page_num)
    if page_num == 5:
        click_next_button()

15
15
15
15
15


In [8]:
for i in range(5):
    for page_num in (range(1, 6)):  
        collect_data_from_page(page_num)
        if page_num == 5:
            click_next_button()

15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15


In [9]:
for page_num in (range(1, 5)):  
    collect_data_from_page(page_num)

15
15
15
5


In [10]:
len(title_list), len(address_list)

(500, 500)

In [11]:
import pandas as pd
df = pd.DataFrame({
    'Title': title_list,
    'Address': address_list
})
df

,Title,Address
0,카페이숲,충남 천안시 서북구 성거읍 남창마을1길 32 1층
1,블렌데렌,"충남 천안시 동남구 먹거리8길 5 1, 2층"
2,그리고스페셜티,충남 천안시 동남구 유량로 104
3,오월의숲,충남 천안시 서북구 세관길 78
4,유얼스,충남 천안시 서북구 불당31길 37
...,...,...
495,메가MGC커피 천안용곡한라점,충남 천안시 동남구 새말1길 55 2층
496,카페다빈제과,충남 천안시 서북구 한들3로 35-11 1층
497,북촌카페,충남 천안시 서북구 직산읍 성진로 422
498,할리스 동천안우체국점,충남 천안시 동남구 청수5로 10 1층


In [28]:
df.to_csv("./data/Cheonan_Cafe_Crawling.csv", index=False)